## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [2]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [5]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000289109E2E60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000028910A104F0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Ravnoor and I am a software engineer")])

AIMessage(content="Hi Ravnoor, it's nice to meet you!\n\nThat's great that you're a software engineer. What kind of software do you enjoy working on?  \n\nI'm always interested in learning more about different areas of software development.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 23, 'total_tokens': 79, 'completion_time': 0.101818182, 'prompt_time': 0.002154638, 'queue_time': 0.083405796, 'total_time': 0.10397282}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b5395259-5514-4868-8453-580a163b69a2-0', usage_metadata={'input_tokens': 23, 'output_tokens': 56, 'total_tokens': 79})

In [7]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Ravnoor and I am a software engineer"),
        AIMessage(content="Hi Ravnoor, it's nice to meet you!\n\nThat's great that you're a software engineer. What kind of software do you enjoy working on?  \n\nI'm always interested in learning more about different areas of software development.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Ravnoor, and you are a software engineer! 😊  \n\nIs there anything else you'd like to tell me about yourself or your work?  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 99, 'total_tokens': 137, 'completion_time': 0.069090909, 'prompt_time': 0.004289376, 'queue_time': 0.109221277, 'total_time': 0.073380285}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-6b20b8ae-c8e7-4c3f-8133-3e7ee56296c3-0', usage_metadata={'input_tokens': 99, 'output_tokens': 38, 'total_tokens': 137})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [8]:
%pip install langchain_community

Note: you may need to restart the kernel to use updated packages.


    tinycss2 (>=1.1.0<1.2) ; extra == 'css'
             ~~~~~~~~^

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [10]:
config={"configurable":{"session_id":"chat1"}}

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Ravnoor and I am a software engineer")],
    config=config
)

In [12]:
response.content

"Hi Ravnoor, it's nice to meet you! 👋\n\nThat's awesome that you're a software engineer.  What kind of software engineering do you specialize in?  \n\nI'm always interested in learning more about what people do! 😊   \n\n"

In [13]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Ravnoor. 😊 \n\nI remember that from our introduction!  Is there anything else I can help you with?  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 95, 'total_tokens': 127, 'completion_time': 0.058181818, 'prompt_time': 0.006038493, 'queue_time': 0.091404241, 'total_time': 0.064220311}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-61c8a76b-a881-473e-8d8d-04738c86f3c5-0', usage_metadata={'input_tokens': 95, 'output_tokens': 32, 'total_tokens': 127})

In [14]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it!\n"

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Singh")],
    config=config1
)
response.content

'Nice to meet you, Singh! \n\nIs there anything I can help you with today? 😊  \n\n'

In [16]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is Singh, you told me earlier!  😄 \n\nHow can I help you further?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [17]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [18]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content="Hello Krish, it's nice to meet you!\n\nI'm happy to help with any questions you have. Just ask away! 😊  What can I do for you today? \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 31, 'total_tokens': 73, 'completion_time': 0.076363636, 'prompt_time': 0.002333267, 'queue_time': 0.083006275, 'total_time': 0.078696903}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-260a973f-028c-4093-a9d6-8529e1548e89-0', usage_metadata={'input_tokens': 31, 'output_tokens': 42, 'total_tokens': 73})

In [19]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [21]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Rav")],
    config=config
)

response

AIMessage(content="Hi Rav! \n\nIt's great to meet you too.  How can I help you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 90, 'total_tokens': 118, 'completion_time': 0.050909091, 'prompt_time': 0.004658984, 'queue_time': 0.084502987, 'total_time': 0.055568075}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d8068306-e22c-47d5-8add-8fd50850bcfa-0', usage_metadata={'input_tokens': 90, 'output_tokens': 28, 'total_tokens': 118})

In [22]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Rav. 😊  \n\nIs there anything else I can help you with?'

In [23]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [26]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Rav")],"language":"Punjabi"})
response.content

'ਸਤਿ ਸ੍ਰੀ ਅਕਾਲ, ਰਾਵ! 😊 \n\nਮੈਂ ਤੁਹਾਡੀ ਮਦਦ ਕਰਨ ਲਈ ਤਿਆਰ ਹਾਂ।  \nतुम क्या जानना चाहो? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [25]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [30]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Rav")],"language":"English"},
    config=config
)
repsonse.content

"Hello Rav! 👋  \n\nIt's nice to meet you. What can I do for you today? 😊  \n"

In [ ]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "English"},
    config=config,
)

In [32]:
response.content

'Your name is Rav.  😊  I remember! \n\n\nIs there anything else I can help you with?\n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [33]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

c:\Code\gen ai\langchain\venv\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\js_ka\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [ ]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal preferences like your favorite ice cream flavor.  \n\nWhat's your favorite ice cream? 😊🍦\n"

In [ ]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2" 😊  \n\n\n\n'

In [ ]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [ ]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [ ]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"